# Springboard Capstone Project #2: 
## Vacation Rental Price Recommender

- **Data source:** A sample of vacation rental (VR) listings from AirDNA
- **Rental market being analyzed:** Cape Coral, FL

## Package & Data Source Imports

In [1]:
import pandas as pd
import json

In [2]:
# read in VR data json file into a DataFrame

# didn't work: vr_data = pd.read_json('../vr_datasets/output.json')

data = json.load(open('../vr_datasets/output.json'))
df = pd.json_normalize(data)

df

,listings_limited,properties,listings_returned,request_info.currency,area_info.geom.code.country,area_info.geom.code.state,area_info.geom.code.region,area_info.geom.code.city,area_info.geom.name.country,area_info.geom.name.state,area_info.geom.name.region,area_info.geom.name.city,area_info.geom.id.country,area_info.geom.id.state,area_info.geom.id.region,area_info.geom.id.city
0,False,"[{'airbnb_property_id': 11046205, 'rating': 0....",1275,native,us,florida,33914,cape-coral,United States,Florida,33914,Cape Coral,1,11,2833,60018


### Now that we've got the data loaded in as a DF and flattened most of the initial nested dictionary structure, let's subset the Properties data into it's own DF. 

In [3]:
properties = df['properties'].to_frame()
print(type(properties))
properties

<class 'pandas.core.frame.DataFrame'>


,properties
0,"[{'airbnb_property_id': 11046205, 'rating': 0...."


### I'm not sure the best way to proceed from how I've parsed the original data file thus far. Going to try another approach for extracting the data...

In [4]:
with open('../vr_datasets/output.json','r') as f:
    data = json.loads(f.read())
    
properties_df = pd.json_normalize(data, record_path=['properties'])

properties_df

,airbnb_property_id,rating,property_type,bathrooms,title,occ,bedrooms,accommodates,adr,img_cover,...,homeaway_property_id,longitude,latitude,m_homeaway_property_id,days_available,id,room_type,regions.zipcode_ids,platforms.airbnb_property_id,platforms.homeaway_property_id
0,11046205,0.0,Villa,2.0,Luxurious Pool Villa with private boat slip,0.404494,3,2,208.3330,https://a0.muscache.com/im/pictures/375a2880-0...,...,None,-82.033230,26.558650,21941ha,89,1753173,Entire home/apt,[2833],11046205.0,21941ha
1,9786136,9.8,House,1.0,Private Friendly Oasis,0.345833,2,4,80.7309,https://a0.muscache.com/im/pictures/a141200a-2...,...,None,-81.995120,26.586620,None,240,1556362,Private room,[2833],9786136.0,None
2,1003435283,10.0,house,2.0,Paradise - New Pool - Canal and Sunshine,0.824742,3,6,136.7500,https://media.vrbo.com/lodging/35000000/340800...,...,1494515,-81.976502,26.544985,None,97,1003435283,Entire home/apt,[2833],NaN,1494515
3,7836717,9.4,House,3.0,7th Heaven - Unique Home Perfect for 2 Families,0.605634,4,8,233.5120,https://a0.muscache.com/im/pictures/03d129d9-8...,...,None,-82.019160,26.571760,9247875ha,71,501312,Entire home/apt,[2833],7836717.0,9247875ha
4,9893241,10.0,Guest suite,1.0,Cape Paradise - Poolside Cabana,0.152381,1,2,189.6940,https://a0.muscache.com/im/pictures/3f0c011f-8...,...,None,-82.004250,26.564580,None,105,1556359,Private room,[2833],9893241.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1270,1000742730,10.0,house,2.0,Beautiful Gulf Access Vacation Home - Pool - H...,0.625000,3,6,238.0000,https://media.vrbo.com/lodging/40000000/399500...,...,872964,-81.977301,26.566971,None,40,1000742730,Entire home/apt,[2833],NaN,872964
1271,1000375573,10.0,House,2.0,"Waterfront Home with heated Pool and Spa, Gulf...",0.495868,3,6,184.4800,https://media.vrbo.com/lodging/35000000/340200...,...,4312377ha,-81.977324,26.582227,None,242,1000375573,Entire home/apt,[2833],NaN,4312377ha
1272,1001904447,10.0,villa,3.5,"Luxury Surfside Home With Waterview, Heated Po...",0.265625,3,6,366.9620,https://media.vrbo.com/lodging/34000000/335700...,...,4760935ha,-82.035941,26.589387,None,128,1001904447,Entire home/apt,[2833],NaN,4760935ha
1273,29833242,8.7,House,3.0,Roelens Vacations - Villa Hideaway - Cape Coral,0.314024,3,6,264.1110,https://a0.muscache.com/im/pictures/a6974f6a-1...,...,None,-82.014170,26.595750,7433783ha,328,11541379,Entire home/apt,[2833],29833242.0,7433783ha


### This is definitely much closer to what I am wanting to work with. 

After two different approaches to loading in the data, my latter approach seems to have been the better route. Based on my research, I can still add other metadata from the original JSON file that I want to include, by passing in a list of keys to the 'meta' parameter of 'json_normalize'. 

In [5]:
properties_df = pd.json_normalize(data, record_path=['properties'], 
                                  meta=['area_info.geom.name.country', 'area_info.geom.name.state', 
                                        'area_info.geom.name.region', 'area_info.geom.name.city'])

properties_df

KeyError: "Try running with errors='ignore' as key 'area_info.geom.name.country' is not always present"

### Based on the error above, going to try same code again but this time will add in errors='ignore'

In [6]:
properties_df = pd.json_normalize(data, record_path=['properties'], 
                                  meta=['area_info.geom.name.country', 'area_info.geom.name.state', 
                                        'area_info.geom.name.region', 'area_info.geom.name.city'],
                                 errors='ignore')

properties_df

,airbnb_property_id,rating,property_type,bathrooms,title,occ,bedrooms,accommodates,adr,img_cover,...,days_available,id,room_type,regions.zipcode_ids,platforms.airbnb_property_id,platforms.homeaway_property_id,area_info.geom.name.country,area_info.geom.name.state,area_info.geom.name.region,area_info.geom.name.city
0,11046205,0.0,Villa,2.0,Luxurious Pool Villa with private boat slip,0.404494,3,2,208.3330,https://a0.muscache.com/im/pictures/375a2880-0...,...,89,1753173,Entire home/apt,[2833],11046205.0,21941ha,NaN,NaN,NaN,NaN
1,9786136,9.8,House,1.0,Private Friendly Oasis,0.345833,2,4,80.7309,https://a0.muscache.com/im/pictures/a141200a-2...,...,240,1556362,Private room,[2833],9786136.0,None,NaN,NaN,NaN,NaN
2,1003435283,10.0,house,2.0,Paradise - New Pool - Canal and Sunshine,0.824742,3,6,136.7500,https://media.vrbo.com/lodging/35000000/340800...,...,97,1003435283,Entire home/apt,[2833],NaN,1494515,NaN,NaN,NaN,NaN
3,7836717,9.4,House,3.0,7th Heaven - Unique Home Perfect for 2 Families,0.605634,4,8,233.5120,https://a0.muscache.com/im/pictures/03d129d9-8...,...,71,501312,Entire home/apt,[2833],7836717.0,9247875ha,NaN,NaN,NaN,NaN
4,9893241,10.0,Guest suite,1.0,Cape Paradise - Poolside Cabana,0.152381,1,2,189.6940,https://a0.muscache.com/im/pictures/3f0c011f-8...,...,105,1556359,Private room,[2833],9893241.0,None,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1270,1000742730,10.0,house,2.0,Beautiful Gulf Access Vacation Home - Pool - H...,0.625000,3,6,238.0000,https://media.vrbo.com/lodging/40000000/399500...,...,40,1000742730,Entire home/apt,[2833],NaN,872964,NaN,NaN,NaN,NaN
1271,1000375573,10.0,House,2.0,"Waterfront Home with heated Pool and Spa, Gulf...",0.495868,3,6,184.4800,https://media.vrbo.com/lodging/35000000/340200...,...,242,1000375573,Entire home/apt,[2833],NaN,4312377ha,NaN,NaN,NaN,NaN
1272,1001904447,10.0,villa,3.5,"Luxury Surfside Home With Waterview, Heated Po...",0.265625,3,6,366.9620,https://media.vrbo.com/lodging/34000000/335700...,...,128,1001904447,Entire home/apt,[2833],NaN,4760935ha,NaN,NaN,NaN,NaN
1273,29833242,8.7,House,3.0,Roelens Vacations - Villa Hideaway - Cape Coral,0.314024,3,6,264.1110,https://a0.muscache.com/im/pictures/a6974f6a-1...,...,328,11541379,Entire home/apt,[2833],29833242.0,7433783ha,NaN,NaN,NaN,NaN


### Hmm... I didn't expect NaNs for each of the 'meta' values. After further researching, it looks like I may have had the formatting wrong for accessing nested keys in the 'meta' parameter..

In [7]:
properties_df = pd.json_normalize(data, record_path=['properties'], 
                                  meta=[['area_info', 'geom', 'name', 'country'],
                                       ['area_info', 'geom', 'name', 'state'],
                                       ['area_info', 'geom', 'name', 'region'],
                                       ['area_info', 'geom', 'name', 'city']],
                                 errors='ignore')

properties_df

,airbnb_property_id,rating,property_type,bathrooms,title,occ,bedrooms,accommodates,adr,img_cover,...,days_available,id,room_type,regions.zipcode_ids,platforms.airbnb_property_id,platforms.homeaway_property_id,area_info.geom.name.country,area_info.geom.name.state,area_info.geom.name.region,area_info.geom.name.city
0,11046205,0.0,Villa,2.0,Luxurious Pool Villa with private boat slip,0.404494,3,2,208.3330,https://a0.muscache.com/im/pictures/375a2880-0...,...,89,1753173,Entire home/apt,[2833],11046205.0,21941ha,United States,Florida,33914,Cape Coral
1,9786136,9.8,House,1.0,Private Friendly Oasis,0.345833,2,4,80.7309,https://a0.muscache.com/im/pictures/a141200a-2...,...,240,1556362,Private room,[2833],9786136.0,None,United States,Florida,33914,Cape Coral
2,1003435283,10.0,house,2.0,Paradise - New Pool - Canal and Sunshine,0.824742,3,6,136.7500,https://media.vrbo.com/lodging/35000000/340800...,...,97,1003435283,Entire home/apt,[2833],NaN,1494515,United States,Florida,33914,Cape Coral
3,7836717,9.4,House,3.0,7th Heaven - Unique Home Perfect for 2 Families,0.605634,4,8,233.5120,https://a0.muscache.com/im/pictures/03d129d9-8...,...,71,501312,Entire home/apt,[2833],7836717.0,9247875ha,United States,Florida,33914,Cape Coral
4,9893241,10.0,Guest suite,1.0,Cape Paradise - Poolside Cabana,0.152381,1,2,189.6940,https://a0.muscache.com/im/pictures/3f0c011f-8...,...,105,1556359,Private room,[2833],9893241.0,None,United States,Florida,33914,Cape Coral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1270,1000742730,10.0,house,2.0,Beautiful Gulf Access Vacation Home - Pool - H...,0.625000,3,6,238.0000,https://media.vrbo.com/lodging/40000000/399500...,...,40,1000742730,Entire home/apt,[2833],NaN,872964,United States,Florida,33914,Cape Coral
1271,1000375573,10.0,House,2.0,"Waterfront Home with heated Pool and Spa, Gulf...",0.495868,3,6,184.4800,https://media.vrbo.com/lodging/35000000/340200...,...,242,1000375573,Entire home/apt,[2833],NaN,4312377ha,United States,Florida,33914,Cape Coral
1272,1001904447,10.0,villa,3.5,"Luxury Surfside Home With Waterview, Heated Po...",0.265625,3,6,366.9620,https://media.vrbo.com/lodging/34000000/335700...,...,128,1001904447,Entire home/apt,[2833],NaN,4760935ha,United States,Florida,33914,Cape Coral
1273,29833242,8.7,House,3.0,Roelens Vacations - Villa Hideaway - Cape Coral,0.314024,3,6,264.1110,https://a0.muscache.com/im/pictures/a6974f6a-1...,...,328,11541379,Entire home/apt,[2833],29833242.0,7433783ha,United States,Florida,33914,Cape Coral


### Much better! There is still some cleaning work to be done, but I think there are some interesting features here to work with for analysis and modeling. 

Next, let's outline our data definitions, check data types, and clean up the main dataframe. 

In [15]:
properties_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1275 entries, 0 to 1274
Data columns (total 26 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   airbnb_property_id              1275 non-null   int64  
 1   rating                          1164 non-null   float64
 2   property_type                   1275 non-null   object 
 3   bathrooms                       1275 non-null   float64
 4   title                           1275 non-null   object 
 5   occ                             1275 non-null   float64
 6   bedrooms                        1275 non-null   int64  
 7   accommodates                    1275 non-null   int64  
 8   adr                             1275 non-null   float64
 9   img_cover                       1275 non-null   object 
 10  reviews                         1275 non-null   int64  
 11  revenue                         1275 non-null   float64
 12  homeaway_property_id            41

### Here are the definitions for the columns, and whether I think it's worth keeping for the analysis (keep/drop; if keeping, will indicate if any transformations or cleaning to be done:
- airbnb_property_id == unique ID for property's airbnb listing (keep)
- rating == avg rating of property by renters (keep); Need to confirm the rating scale, but looks like 1 - 10
- property_type == categorical value of property types (keep); need to better determine categories
- bathrooms == count of total bathrooms in the property (keep)
- title == the title text from the rental listing (keep); might be useful at some point, but may eventually drop
- occ == occupancy rate (keep); assumption is that this is avg occupancy rate for the rental since it's been listed
- bedrooms == count of total bedrooms in the property (keep)
- accommodates == count of max guests a property can accomodate (keep)
- adr == Average Daily Revenue (keep); Will rename field to 'avg_daily_revenue' make it's meaning more clear
- img_cover == cover image for the listing (drop)
- reviews == not sure, need to inspect
- revenue == not sure, need to inspect                        
- homeaway_property_id == unique ID for property's Homeaway listing (keep); what is diff to 'm_homeaway_property_id'? 
- longitude (drop)    
- latitude (drop)
- m_homeaway_property_id == not sure what diff is to 'homeaway_property_id'; need to understand before keep/drop
- days_available = assumption: how many days the property was actively listed for rent in a 1 year period OR how many days it was vacant in a 1 year period; need to clarify
- id ==not sure diff here from rental listing site IDs, need to inspect
- room_type == categorical value describing the property's room type (keep)
- regions.zipcode_ids (drop)
- platforms.airbnb_property_id == not sure what diff is to 'airbnb_property_id'; need to understand before keep/drop
- platforms.homeaway_property_id == not sure what diff is to 'homeaway_property_id'; need to understand before keep/drop 
- area_info.geom.name.country == country, duplicated for each record; will rename column to 'country' 
- area_info.geom.name.state == state, duplicated for each record; will rename column to 'state'  
- area_info.geom.name.region == region, duplicated for each record; will rename column to 'region' 
- area_info.geom.name.city == city, duplicated for each record; will rename column to 'city'    




In [18]:
# Drop columns that I'm confident I won't need for analysis and modeling

properties_df = properties_df.drop(['img_cover','longitude','latitude','regions.zipcode_ids'], axis=1)
print(properties_df)

      airbnb_property_id  rating property_type  bathrooms  \
0               11046205     0.0         Villa        2.0   
1                9786136     9.8         House        1.0   
2             1003435283    10.0         house        2.0   
3                7836717     9.4         House        3.0   
4                9893241    10.0   Guest suite        1.0   
...                  ...     ...           ...        ...   
1270          1000742730    10.0         house        2.0   
1271          1000375573    10.0         House        2.0   
1272          1001904447    10.0         villa        3.5   
1273            29833242     8.7         House        3.0   
1274            46356115    10.0         House        2.0   

                                                  title       occ  bedrooms  \
0           Luxurious Pool Villa with private boat slip  0.404494         3   
1                                Private Friendly Oasis  0.345833         2   
2              Paradise - New 

In [20]:
properties_df.columns

Index(['airbnb_property_id', 'rating', 'property_type', 'bathrooms', 'title',
       'occ', 'bedrooms', 'accommodates', 'adr', 'reviews', 'revenue',
       'homeaway_property_id', 'm_homeaway_property_id', 'days_available',
       'id', 'room_type', 'platforms.airbnb_property_id',
       'platforms.homeaway_property_id', 'area_info.geom.name.country',
       'area_info.geom.name.state', 'area_info.geom.name.region',
       'area_info.geom.name.city'],
      dtype='object')

In [21]:
#Clean up column names
properties_df = properties_df.rename(columns={"occ":"occupancy_rate", "adr":"avg_daily_revenue", 
                                              "area_info.geom.name.country":"country", 
                                              "area_info.geom.name.state":"state", 
                                              "area_info.geom.name.region":"region", 
                                              "area_info.geom.name.city":"city"})

In [22]:
properties_df.columns

Index(['airbnb_property_id', 'rating', 'property_type', 'bathrooms', 'title',
       'occupancy_rate', 'bedrooms', 'accommodates', 'avg_daily_revenue',
       'reviews', 'revenue', 'homeaway_property_id', 'm_homeaway_property_id',
       'days_available', 'id', 'room_type', 'platforms.airbnb_property_id',
       'platforms.homeaway_property_id', 'country', 'state', 'region', 'city'],
      dtype='object')

In [25]:
# Compare IDs to clarify if any columns are duplicates and better understand differences of similarly title columns
id_df = properties_df.loc[:,['airbnb_property_id', 'platforms.airbnb_property_id', 'homeaway_property_id', 'm_homeaway_property_id', 
                         'platforms.homeaway_property_id']]
id_df

,airbnb_property_id,platforms.airbnb_property_id,homeaway_property_id,m_homeaway_property_id,platforms.homeaway_property_id
0,11046205,11046205.0,None,21941ha,21941ha
1,9786136,9786136.0,None,None,None
2,1003435283,NaN,1494515,None,1494515
3,7836717,7836717.0,None,9247875ha,9247875ha
4,9893241,9893241.0,None,None,None
...,...,...,...,...,...
1270,1000742730,NaN,872964,None,872964
1271,1000375573,NaN,4312377ha,None,4312377ha
1272,1001904447,NaN,4760935ha,None,4760935ha
1273,29833242,29833242.0,None,7433783ha,7433783ha


In [27]:
id_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1275 entries, 0 to 1274
Data columns (total 5 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   airbnb_property_id              1275 non-null   int64  
 1   platforms.airbnb_property_id    857 non-null    float64
 2   homeaway_property_id            418 non-null    object 
 3   m_homeaway_property_id          429 non-null    object 
 4   platforms.homeaway_property_id  847 non-null    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 49.9+ KB


### It looks like its safe to drop 'homeaway_property_id' and 'm_homeaway_property_id', since these values are included in "platforms.homeaway_property_id".

### It's not as clear yet which of the airbnb_property_id columns are more accurate and worth keeping.

In [29]:
properties_df = properties_df.drop(columns=['homeaway_property_id', 'm_homeaway_property_id'], axis=1)
properties_df.head()

,airbnb_property_id,rating,property_type,bathrooms,title,occupancy_rate,bedrooms,accommodates,avg_daily_revenue,reviews,revenue,days_available,id,room_type,platforms.airbnb_property_id,platforms.homeaway_property_id,country,state,region,city
0,11046205,0.0,Villa,2.0,Luxurious Pool Villa with private boat slip,0.404494,3,2,208.3330,0,7500.00,89,1753173,Entire home/apt,11046205.0,21941ha,United States,Florida,33914,Cape Coral
1,9786136,9.8,House,1.0,Private Friendly Oasis,0.345833,2,4,80.7309,62,6700.67,240,1556362,Private room,9786136.0,None,United States,Florida,33914,Cape Coral
2,1003435283,10.0,house,2.0,Paradise - New Pool - Canal and Sunshine,0.824742,3,6,136.7500,14,10940.00,97,1003435283,Entire home/apt,NaN,1494515,United States,Florida,33914,Cape Coral
3,7836717,9.4,House,3.0,7th Heaven - Unique Home Perfect for 2 Families,0.605634,4,8,233.5120,17,10041.00,71,501312,Entire home/apt,7836717.0,9247875ha,United States,Florida,33914,Cape Coral
4,9893241,10.0,Guest suite,1.0,Cape Paradise - Poolside Cabana,0.152381,1,2,189.6940,4,3035.11,105,1556359,Private room,9893241.0,None,United States,Florida,33914,Cape Coral
